In [1]:
import os, sys
project_dir = os.path.join(os.getcwd(),'./')
if project_dir not in sys.path:
    sys.path.append(project_dir)

medmnist_dir = os.path.join(project_dir, 'modules/MedMNIST')
if medmnist_dir not in sys.path:
    sys.path.append(medmnist_dir)

ipdl_dir = os.path.join(project_dir, 'modules/IPDL')
if ipdl_dir not in sys.path:
    sys.path.append(ipdl_dir)    

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from experiment.autoencoder import PCAE

pcae_exp = PCAE() 

# Pretrain (FMNIST)

In [3]:
from torchvision import datasets
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import DataLoader

transforms = Compose([Resize((64, 64)), ToTensor()])

train_set = datasets.FashionMNIST("dataset", train=True, transform=transforms, download=True)
test_set = datasets.FashionMNIST("dataset", train=False, transform=transforms, download=True)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = DataLoader(test_set, batch_size=128, shuffle=False)

In [4]:
from torch.utils.tensorboard import SummaryWriter

tb_writer = SummaryWriter('logs/{}/AE_FMNIST'.format(pcae_exp.model_name))
pcae_exp.train(train_loader, val_loader, tb_writer, n_epoch=250)

100%|██████████| 250/250 [46:25<00:00, 11.14s/epoch, model=PCAE, tls=0.0012, vls=0.0012]


In [7]:
df = pcae_exp.ip.to_df()
df.to_pickle('data/PCAE/MI/AE_FMNIST.pkl')

In [10]:
import torch
torch.save(pcae_exp.model.state_dict(), 'data/PCAE/weights/AE/FMNIST.pt')

In [8]:
test = pd.read_pickle('data/PCAE/MI/AE_FMNIST.pkl')
test

Layer 1             Layer 2             Layer 3             Layer 4  \
          Ixt       Ity       Ixt       Ity       Ixt       Ity       Ixt   
0    6.229723  0.000039  0.607034  0.000039  0.011361  0.000039  0.000515   
1    6.991510  6.937258  6.942041  6.896255  6.626875  6.599549  6.938232   
2    6.990592  6.987859  6.903050  6.900978  6.145448  6.144490  6.890813   
3    6.992561  6.991616  6.909446  6.908758  6.259130  6.258812  6.872007   
4    6.993776  6.993389  6.903475  6.903201  6.244779  6.244645  6.828091   
..        ...       ...       ...       ...       ...       ...       ...   
245  6.996223  6.996218  6.842934  6.842930  5.459518  5.459517  6.302378   
246  6.996228  6.996223  6.843458  6.843455  5.467689  5.467688  6.310882   
247  6.996224  6.996218  6.843156  6.843153  5.465965  5.465964  6.307345   
248  6.996221  6.996216  6.842608  6.842606  5.466276  5.466275  6.311301   
249  6.996222  6.996215  6.841994  6.841990  5.464746  5.464744  6.306072   

                Layer 5            
          Ity       Ixt       Ity  
0    0.000039  0.000072  0.000039  
1    6.897676  6.991201  6.939475  
2    6.888764  6.997214  6.994377  
3    6.871360  6.997864  6.996881  
4    6.827849  6.998289  6.997890  
..        ...       ...       ...  
245  6.302378  6.990791  6.990786  
246  6.310881  6.991001  6.990997  
247  6.307343  6.990913  6.990908  
248  6.311299  6.991041  6.991035  
249  6.306070  6.990851  6.990844  

[250 rows x 10 columns]

In [ ]:
import medmnist  
from medmnist import INFO

data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [ ]:
import torchvision.transforms as transforms

data_transform = transforms.Compose([
    transforms.Resize((64, 64)), 
    transforms.ToTensor(),
    # transforms.Normalize(mean=[.5], std=[.5])
])

train_dataset = DataClass(split='train', transform=data_transform, download=download)
eval_dataset = DataClass(split='test', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)


## Reduce Train dataset

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

x = train_dataset.imgs
y = train_dataset.labels

sampling_strategies = {0: 128, 1: 128}
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategies, random_state=123)
X_resampled, y_resampled = undersampler.fit_resample(x.reshape((x.shape[0], -1)), y.flatten())

train_dataset.imgs = X_resampled.reshape((-1,x.shape[1],x.shape[2]))
train_dataset.labels = y_resampled.reshape((-1, y.shape[1]))

In [ ]:
print(len(train_dataset))

### Reduce validation dataset

In [ ]:
x = eval_dataset.imgs
y = eval_dataset.labels

sampling_strategies = {0: 16, 1: 16}
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategies, random_state=123)
X_resampled, y_resampled = undersampler.fit_resample(x.reshape((x.shape[0], -1)), y.flatten())

eval_dataset.imgs = X_resampled.reshape((-1,x.shape[1],x.shape[2]))
eval_dataset.labels = y_resampled.reshape((-1, y.shape[1]))

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(dataset=eval_dataset, batch_size=32, shuffle=True)

from torch.utils.tensorboard import SummaryWriter

tb_writer = SummaryWriter('log/PCAE')

exp.train(train_loader, eval_loader, tb_writer=tb_writer)

In [ ]:
x, y = next(iter(train_loader))

model = exp.model
model.encode(x.to(exp.device)).shape

In [ ]:
print(len(eval_dataset))

In [ ]:
print(len(test_dataset))

In [ ]:
x.reshape((x.shape[0], -1)).shape

In [ ]:
x.to_numpy()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

sampling_strategies = {0: 128, 1: 128}
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategies, random_state=123)
X_resampled, y_resampled = undersampler.fit_resample(x.reshape((x.shape[0], -1)), y.flatten())

In [ ]:
train_dataset.imgs = X_resampled.reshape((-1,x.shape[1],x.shape[2]))
train_dataset.labels = y_resampled.reshape((-1, y.shape[1]))

In [ ]:
train_dataset

In [ ]:
train_dataset.imgs.shape

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
for i, n in train_loader:
    print(n.shape)

In [ ]:
122 + 45 + 74

In [ ]:
train_dataset